In [0]:
from google.colab import drive
import os
drive.mount('/content/gdrive')#, force_remount=True)
curDir =('/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project')
os.chdir(curDir)
!cat /content/gdrive/My\ Drive/Colab\ Notebooks/DeepMovement_project

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
cat: '/content/gdrive/My Drive/Colab Notebooks/DeepMovement_project': Is a directory


In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0




# 1.   Imports




In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import time
import pickle
from random import shuffle
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation#, SpatialDropout1D
from keras.optimizers import SGD, Adam
# from google.colab import files
# src = list(files.upload().values())[0]
# open('UsedFunctions.py','wb').write(src)
from UsedFunctions import modelBuild
%load_ext autoreload
%autoreload 2


Using TensorFlow backend.


In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

# 2. Methods

In [0]:
# In[2] Methods
def Regulate(framesInSeq):
    rem=[]
    for j in range(len(framesInSeq)):
        if framesInSeq[j] <= 7:
            if len(framesInSeq) <= 1:
                print("Error, very small file")
                break
            if j == 0 and (j+1) < len(framesInSeq):
                framesInSeq[j+1] += framesInSeq[j]
            elif j == len(framesInSeq)-1 and (j-1) >= 0:
                framesInSeq[j-1] += framesInSeq[j]
            else:
                if framesInSeq[j+1] < framesInSeq[j-1]:
                    framesInSeq[j+1] += framesInSeq[j]  
                else: framesInSeq[j-1] += framesInSeq[j]
            rem.append(j)
    framesInSeq=np.delete(framesInSeq,rem)
    return framesInSeq

def ReadData():
    """ ----- Very important to note the pathes ----- """
    # Getting training data
    df = pd.read_csv(os.path.join('.','Output','Activity_train.csv')) #read data
    df.dropna(axis=0, how='any', thresh=76, subset=None, inplace=True)
    # Getting consistant sequence of frames
    fr = df.copy()['Frame']
    target = df.copy()['Human']
    
    #dropping usless columns and columns with all their values are 0
    dropped = ['Human', 'V_title', 'Frame', 'L Foot_x_PCM', 'L Foot_y_PCM', 'R Foot_L',	'L Foot_L',	'Head & Neck_R',	'Trunk_R',	'R Upper Arm_R',	'L Upper Arm_R',	'R Forearm_R',	'L Forearm_R',	'R Hand_R',	'L Hand_R',	'R Thigh_R',	'L Thigh_R',	'R Shank_R',	'L Shank_R',	'R Foot_R',	'L Foot_R']
    df.drop(columns=dropped, inplace=True)
    
    seq_count = np.zeros(fr.shape, dtype=int)
    for i in range(1, len(fr.index)):
        # assuming if there is > 30 frames gap, the scene is changed
        if np.abs(fr.values[i-1] - fr.values[i]) > 30:
            seq_count[i] = seq_count[i-1] + 1
        else:
            seq_count[i] = seq_count[i-1]
    framesInSeq = np.bincount(seq_count)
    
    framesInSeq = Regulate(framesInSeq)
    
    maxSeq = framesInSeq.max() #// 13
        
    nan_locations = np.isnan(np.float_(df.values)) # get locations of nan values to set to 0 later
    df.fillna(0, inplace=True) # convert NaN values to zeros
    
    array = df.values #conver df to array
    X = np.array(array, dtype='float')
    #target = array[:, 0] #split target data
    l = list(set(target.copy()))
    labelencoder = LabelEncoder()
    target = labelencoder.fit_transform(target)
    
    # get unique labels 
    unique_labels = dict(zip(labelencoder.fit_transform(l), l))
    
    #X = np.delete(X, np.s_[:1], 1) #drop target data 
    #X = np.float64(X)
    Y = np.array(target, dtype='int')
    #Y=Y.astype('int') # issue
    
    # Feature scaling 
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X[nan_locations]=0 # set location of nans to 0
    
    Data_X = []
    Data_Y = []
    Num = 0
    ind = 0
    for cLen in framesInSeq:
        #if clen > maxSeq:
        #    clen /= 3
        for i in range((cLen // maxSeq) + 1 * ((cLen % maxSeq)!=0)): # in normal situations i is always =0
            x = []
            for j in range(maxSeq):
                if (maxSeq*i + j) >= cLen:
                    x = np.tile(x, (maxSeq//len(x)+1, 1))[:maxSeq].tolist()
                    break
                else:
                    x.append(list(X[Num + maxSeq*i + j]))
                    y = Y[Num + maxSeq*i + j]
            Data_X.append(x)
            Data_Y.append(y) 
        Num += cLen
        ind += 1
    Data_Y = to_categorical(Data_Y) # convert to one hot
    maxSeq=50 ##@!
    return np.array(Data_X, dtype='float32')[:,:maxSeq], np.array(Data_Y, dtype='float32'), unique_labels, maxSeq

def modelBuild2(input_shape, lstm_out, dense_out):
    NumLyrs = len(lstm_out)
    NumDns = len(dense_out)
    sentence_indices = Input(input_shape)
    # The returned output should be a batch of sequences.
    if NumLyrs > 1:
        X = LSTM(lstm_out[0], return_sequences=True, dropout=0.4, recurrent_dropout=0.5)(sentence_indices)
        if NumLyrs>2:
            # Propagate X trough another LSTM layer with lstm_out-dimensional hidden state
            X = LSTM(lstm_out[1], return_sequences=True, dropout=0.4, recurrent_dropout=0.5)(X)
            if NumLyrs>3:
                # Propagate X trough another LSTM layer with lstm_out-dimensional hidden state
                X = LSTM(lstm_out[2], return_sequences=True, dropout=0.4, recurrent_dropout=0.5)(X)
    else:
        X = sentence_indices
    X = LSTM(lstm_out[-1], return_sequences=False, dropout=0.4, recurrent_dropout=0.5)(X)
    # Propagate X through a Dense layer with softmax activation.
    if NumDns>1:
        X = Dense(dense_out[0], activation='relu')(X)
        X = Dropout(0.4)(X)
        if NumDns>2:
            X = Dense(dense_out[1], activation='relu')(X)
            X = Dropout(0.4)(X)
    X = Dense(dense_out[-1])(X) # Number of classes
    # Add a softmax activation
    X = Activation('softmax')(X)
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    return model


# 3. Training initialization variables

In [0]:
beg = time.time()
# Variables
# sequence_size = 3
batch_size = 128
lr = 5e-4
decay = 1e-7
epochs = 1000
lstm_out = [128,64,32]
dense_layer = 128

NameError: ignored

# 4. Data manipulation

In [0]:
#reading data 
Trainpath = os.path.join(curDir, 'TrainData')
Data_X, Data_Y, classes, sequence_size = ReadData()
# shuffling data
Z= list(zip(Data_X, Data_Y))
shuffle(Z)
Data_X, Data_Y = zip(*Z)
Data_X, Data_Y = np.array(Data_X), np.array(Data_Y)
X_train, X_test, y_train, y_test = train_test_split(Data_X, Data_Y, test_size=0.20, random_state=42)


In [0]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(classes)

(913, 50, 79) (913, 3)
(229, 50, 79) (229, 3)
{2: 'walking', 0: 'jogging', 1: 'running'}


# 5. Training

In [0]:

with tf.device('/gpu:0'):
    # Building the model
    model = modelBuild2(X_train.shape[1:], lstm_out, [dense_layer, len(classes)])
    print(model.summary())
    opt = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    # Saving weights of the model after every epoch in a given directory
    modelspath=os.path.join(curDir,'ModelData')
    if not os.path.exists(modelspath):
        os.makedirs(modelspath)
    # Preparing callbacks during training
    temppath=os.path.join(curDir,'ModelData', 'Activity_P1')
    if not os.path.exists(temppath):
        os.makedirs(temppath)
    filepath=os.path.join(temppath,'weights-{epoch:02d}-{val_acc:.2f}.hdf5') ##@
    earlystop = EarlyStopping(monitor='val_loss', patience=500, verbose=1, restore_best_weights=True)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_weights_only=True, save_best_only=True)
    # using tensorbard to visualize the training and view learning curves
    # to know when to stop and choose which epoch as the best, while the code is running 
    # run the following command in your terminal while pointing to the script directory
    tbCallBack = TensorBoard(log_dir='log', histogram_freq=0, write_graph=False, write_images=True)
    model.load_weights(os.path.join(temppath, 'weights-703-0.80.hdf5'))
    # training the model and using validation data to validate the parameters (overfitting and so on..)
    history = model.fit(X_train, y_train, validation_data=(X_test,y_test),
                        epochs = epochs, batch_size = batch_size, shuffle=True,
                        verbose = 2, callbacks=[tbCallBack,earlystop])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 79)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 128)           106496    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 64)            49408     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
______

KeyboardInterrupt: ignored

In [0]:
plot_graphs(history, 'acc')
plot_graphs(history, 'loss')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1806af2178f8>", line 1, in <module>
    plot_graphs(history, 'acc')
NameError: name 'history' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultra

NameError: ignored

# 6. Save necessary files

In [0]:


model.save(os.path.join(temppath,'my_model.hdf5'))
model.save_weights(os.path.join(temppath,'my_model_weights.hdf5'))
with open(os.path.join(temppath,'model_architecture.json'), 'w') as f:
    f.write(model.to_json())
# Saving some variables that might be used during testing
filename = os.path.join(temppath, 'LSTM_model_variables.sav')
with open(filename, 'wb') as handle:
    pickle.dump(classes, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(sequence_size, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(batch_size, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(lr, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(decay, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(lstm_out, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(dense_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 7. Evaluating

In [0]:


loss, acc = model.evaluate(X_test, y_test)
print()
print("Test accuracy = ", acc)

# Prediction
# model.load_weights(os.path.join(temppath, 'weights-828-0.81.hdf5'))
pred = model.predict(X_test)
#print(confusion_matrix(y_test.argmax(axis=1), pred.argmax(axis=1)))
y_actu = pd.Series(y_test.argmax(axis=1), name='Actual')
y_pred = pd.Series(pred.argmax(axis=1), name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
print(df_confusion)
print('Total time taken is {:.3f} min'.format((time.time()-beg)/60))

## Tests

In [0]:
from IPython.display import clear_output

for i in range(10):
    clear_output()
    print("Hello World!")
sleep(2)
clear_output()

In [0]:
# df = pd.read_csv(os.path.join('.','Output','Data_train.csv')) #read data
# df = df.append(pd.read_csv(os.path.join('.','Output','Humans_train.csv'))) #read data
# df = df.append(pd.read_csv(os.path.join('.','Output','Humans_train_lite.csv')).replace('P1', 'P11'))
# df = df.append(pd.read_csv(os.path.join('.','Output','Humans_train_added.csv')))
df = pd.read_csv(os.path.join('.','Output','Activity_train.csv'))
df = df.fillna(0) #conver NaN values to zeros
# Getting consistant sequence of frames
fr = df.copy()['Frame']
seq_count = np.zeros(fr.shape, dtype=int)
for i in range(1, len(fr.index)):
    # assuming if there is > 20 frames gap, the scene is changed
    if np.abs(fr.values[i-1] - fr.values[i]) > 30:
        seq_count[i] = seq_count[i-1] + 1
    else:
        seq_count[i] = seq_count[i-1]
framesInSeq = np.bincount(seq_count)
maxSeq = framesInSeq.max()
print(maxSeq)

In [0]:
print(framesInSeq.mean())
print(np.median(framesInSeq))
print(np.bincount(framesInSeq).argmax())
print(sorted(framesInSeq))
# [1, 1, 3, 5, 14, 14, 32, 40, 42, 47, 64, 72, 83, 85, 90, 99, 108, 120, 120, 121, 126, 129, 130, 144, 148, 281, 297, 325, 385, 861]
# len([0]*X.shape[1]) 

In [5]:
import numpy as np
te = np.array(list(range(2,9))+[5])
te = te / te.sum()
print((-te*np.log2(te)).sum())
print((te*[5,5,4,4,3,3,1,4]).sum())

2.893542397143181
3.1999999999999997


In [8]:
te2 = np.array([45,13,12,16,9,5,2,15])
te2 = te2 / te2.sum()
print((-te2*np.log2(te2)).sum())
print((te2*[1,4,4,3,4,5,5,3]).sum())

2.5712110435076028
2.641025641025641


In [10]:
x=np.array([0.5,0.25,1/8,1/8])
print((-x*np.log2(x)).sum())
print((x*[1,2,3,3]).sum())

1.75
1.75
